### DEMO 2A

#### Matriz de cambio.

In [ ]:
from docplex.cp.model import CpoModel, transition_matrix
import docplex.cp.utils_visu as visu

m = CpoModel()

mt = [[0, 0, 10, 10, 0, 0,15],
     [0,0, 10, 10, 0, 0, 15],
     [10,10, 0, 0, 0, 0, 15],
     [50,50, 10, 0, 0, 0,15],
     [100,100,15,15,0, 0,15],
     [100,100,50,50,15,0,15],
     [100,100,50,50,15,15,0]]


matriz = transition_matrix(mt)

p1 = m.interval_var(size=8,  name='P1')
p2 = m.interval_var(size=10, name='P2')
p3 = m.interval_var(size=6,  name='P3')
p4 = m.interval_var(size=12, name='P4')
p5 = m.interval_var(size=9,  name='P5')
p6 = m.interval_var(size=14, name='P6')

t = [p1, p2, p3, p4, p5, p6]

tipos = [1, 3, 6, 5, 2, 0]

s = m.sequence_var(t, types=tipos, name='S')
        
m.add(m.no_overlap(s, matriz, True))

m.add( m.minimize(   m.max_of([m.end_of(p1), m.end_of(p2), m.end_of(p3), m.end_of(p4), m.end_of(p5), m.end_of(p6)] )))

s = m.solve()

s.print_solution()

if s:
    visu.panel()
    visu.interval(s.get_var_solution(p1), 1, 'P1_'+str(tipos[0]))
    visu.interval(s.get_var_solution(p2), 2, 'P2_'+str(tipos[1]))
    visu.interval(s.get_var_solution(p3), 3, 'P3_'+str(tipos[2]))
    visu.interval(s.get_var_solution(p4), 4, 'P4_'+str(tipos[3]))
    visu.interval(s.get_var_solution(p5), 5, 'P5_'+str(tipos[4]))
    visu.interval(s.get_var_solution(p6), 6, 'P6_'+str(tipos[5]))

    visu.matrix(name="M1", matrix=mt)
    
    visu.show(origin=0, horizon=30)



### DEMO 2B

#### Función de estados.

In [ ]:
from docplex.cp.model import CpoModel, transition_matrix
import docplex.cp.utils_visu as visu

m2 = CpoModel()

matriz = [[0, 12, 5],
          [12, 0 ,10],
          [5, 10, 0]]

ms = transition_matrix(matriz)

sf = m2.state_function(trmtx=ms, name='sf')


p1 = m2.interval_var(size=8,  name='P1')
p2 = m2.interval_var(size=10, name='P2')
p3 = m2.interval_var(size=6,  name='P3')


m2.add(m2.always_equal(sf, p1, 0))
m2.add(m2.always_equal(sf, p2, 1))
m2.add(m2.always_equal(sf, p3, 2))

m2.add( m2.minimize(   m2.max_of([m2.end_of(p1), m2.end_of(p2), m2.end_of(p3)] )))

s2 = m2.solve()

s2.print_solution()

# 2 0 1

if s2:
    visu.panel()
    visu.interval(s2.get_var_solution(p1), 'pink', 'P1[0]')
    visu.interval(s2.get_var_solution(p2), 'lightblue', 'P2[1]')
    visu.interval(s2.get_var_solution(p3), 'lightgreen', 'P3[2]')

    # Gráfico de utilización de recursos
    #visu.sequence(name=sf.get_name(), segments=s2.get_var_solution(sf))
    visu.panel(name="función de estados")

    ff = s2.get_var_solution(sf)
    a = []
    for i in range( 1, len(ff.get_value())-1):
        if ff.get_value()[i][2]==-1:
            a.append((ff.get_value()[i][0], ff.get_value()[i][1], ant, ff.get_value()[i+1][2]))
        else:    
            a.append((ff.get_value()[i][0], ff.get_value()[i][1],ff.get_value()[i][2]))
            ant = ff.get_value()[i][2]


    #visu.function(segments=s2.get_var_solution(sf), style='segment', origin=0, horizon=30, color='khaki')     
    visu.function(segments=a, style='segment', origin=0, horizon=40, color='red')     


    visu.panel(name="Matriz de tiempos")
    visu.matrix(name="M1", matrix=matriz)
    


    visu.show(origin=0, horizon=30)

